In [ ]:
# NERVA Quantum FX Dynamics Engine - ALPHA CLEARANCE
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

class QuantumFXEngine:
    def __init__(self):
        self.fx_models = {}
        self.volatility_models = {}
        self.intervention_detector = None
        
        # Professional color scheme
        self.colors = {
            'primary': '#1f77b4',
            'secondary': '#ff7f0e', 
            'success': '#2ca02c',
            'warning': '#d62728',
            'info': '#9467bd'
        }
        
        pio.templates.default = 'plotly_white'
    
    def load_fx_data(self):
        """Load and prepare FX data"""
        # Create synthetic FX data for demonstration
        np.random.seed(42)
        dates = pd.date_range('2020-01-01', periods=1000, freq='D')
        
        # USD/KES with realistic dynamics
        base_rate = 110
        trend = np.cumsum(np.random.normal(0, 0.5, 1000)) * 0.1
        volatility = np.abs(np.sin(np.arange(1000) * 0.02)) * 2 + 1
        noise = np.random.normal(0, 1, 1000) * volatility
        
        usd_kes = base_rate + trend + noise
        
        # Other currency pairs
        eur_kes = usd_kes * (1.1 + np.random.normal(0, 0.05, 1000))
        gbp_kes = usd_kes * (1.25 + np.random.normal(0, 0.03, 1000))
        
        fx_data = pd.DataFrame({
            'date': dates,
            'USD_KES': usd_kes,
            'EUR_KES': eur_kes,
            'GBP_KES': gbp_kes,
            'volatility': volatility
        })
        
        # Add technical indicators
        for pair in ['USD_KES', 'EUR_KES', 'GBP_KES']:
            fx_data[f'{pair}_return'] = fx_data[pair].pct_change()
            fx_data[f'{pair}_sma_20'] = fx_data[pair].rolling(20).mean()
            fx_data[f'{pair}_volatility'] = fx_data[f'{pair}_return'].rolling(20).std() * np.sqrt(252)
        
        print("FX Data loaded successfully")
        print(f"Data shape: {fx_data.shape}")
        print(f"Date range: {fx_data['date'].min()} to {fx_data['date'].max()}")
        
        return fx_data
    
    def analyze_volatility_clustering(self, fx_data, pair='USD_KES'):
        """Analyze volatility clustering patterns"""
        returns = fx_data[f'{pair}_return'].dropna()
        
        # Calculate ARCH effects
        squared_returns = returns ** 2
        autocorr = squared_returns.autocorr()
        
        # Volatility regimes
        vol_high = returns.std() * 1.5
        vol_low = returns.std() * 0.5
        
        high_vol_periods = np.abs(returns) > vol_high
        low_vol_periods = np.abs(returns) < vol_low
        
        clustering_analysis = {
            'autocorrelation': autocorr,
            'high_vol_periods': high_vol_periods.sum(),
            'low_vol_periods': low_vol_periods.sum(),
            'volatility_persistence': squared_returns.rolling(30).mean().std()
        }
        
        print(f"Volatility Clustering Analysis for {pair}:")
        print(f"  Autocorrelation: {autocorr:.4f}")
        print(f"  High volatility periods: {high_vol_periods.sum()}")
        print(f"  Volatility persistence: {clustering_analysis['volatility_persistence']:.4f}")
        
        return clustering_analysis
    
    def detect_intervention_points(self, fx_data, pair='USD_KES', threshold=3):
        """Detect potential central bank intervention points"""
        returns = fx_data[f'{pair}_return'].dropna()
        
        # Z-score based detection
        z_scores = np.abs((returns - returns.mean()) / returns.std())
        potential_interventions = z_scores > threshold
        
        # Rate of change acceleration
        rate_change = fx_data[pair].diff().diff()
        acceleration_threshold = rate_change.std() * 2
        acceleration_events = np.abs(rate_change) > acceleration_threshold
        
        intervention_points = potential_interventions | acceleration_events
        
        print(f"Intervention Detection for {pair}:")
        print(f"  Potential intervention points: {intervention_points.sum()}")
        print(f"  Percentage of observations: {intervention_points.mean()*100:.2f}%")
        
        return intervention_points, z_scores
    
    def create_fx_dashboard(self, fx_data):
        """Create comprehensive FX analysis dashboard"""
        
        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=[
                'USD/KES Exchange Rate', 'Cross-Currency Correlations',
                'Volatility Analysis', 'Intervention Detection',
                'Return Distributions', 'Technical Indicators'
            ],
            specs=[[{}, {}],
                   [{}, {}],
                   [{}, {}]]
        )
        
        # 1. Main FX rate
        fig.add_trace(
            go.Scatter(
                x=fx_data['date'], 
                y=fx_data['USD_KES'],
                mode='lines',
                name='USD/KES',
                line=dict(color=self.colors['primary'], width=2)
            ), row=1, col=1
        )
        
        # Add SMA
        fig.add_trace(
            go.Scatter(
                x=fx_data['date'],
                y=fx_data['USD_KES_sma_20'],
                mode='lines',
                name='20-day SMA',
                line=dict(color=self.colors['secondary'], dash='dash')
            ), row=1, col=1
        )
        
        # 2. Cross-currency correlations (heatmap-style)
        corr_data = fx_data[['USD_KES', 'EUR_KES', 'GBP_KES']].corr()
        
        fig.add_trace(
            go.Heatmap(
                z=corr_data.values,
                x=corr_data.columns,
                y=corr_data.columns,
                colorscale='RdBu',
                zmid=0,
                showscale=False
            ), row=1, col=2
        )
        
        # 3. Volatility time series
        fig.add_trace(
            go.Scatter(
                x=fx_data['date'],
                y=fx_data['USD_KES_volatility'],
                mode='lines',
                name='Volatility',
                line=dict(color=self.colors['warning'], width=2)
            ), row=2, col=1
        )
        
        # 4. Intervention points
        intervention_points, z_scores = self.detect_intervention_points(fx_data)
        
        fig.add_trace(
            go.Scatter(
                x=fx_data['date'],
                y=z_scores,
                mode='lines',
                name='Z-Scores',
                line=dict(color=self.colors['info'])
            ), row=2, col=2
        )
        
        # Add intervention threshold
        fig.add_hline(y=3, line_dash="dash", line_color="red", row=2, col=2)
        fig.add_hline(y=-3, line_dash="dash", line_color="red", row=2, col=2)
        
        # 5. Return distribution
        returns = fx_data['USD_KES_return'].dropna()
        
        fig.add_trace(
            go.Histogram(
                x=returns,
                nbinsx=50,
                name='Return Distribution',
                marker_color=self.colors['success'],
                opacity=0.7
            ), row=3, col=1
        )
        
        # 6. Technical momentum
        momentum = fx_data['USD_KES'] - fx_data['USD_KES_sma_20']
        
        fig.add_trace(
            go.Scatter(
                x=fx_data['date'],
                y=momentum,
                mode='lines',
                name='Momentum',
                line=dict(color=self.colors['primary'])
            ), row=3, col=2
        )
        
        fig.add_hline(y=0, line_dash="solid", line_color="black", row=3, col=2)
        
        fig.update_layout(
            title_text="NERVA Quantum FX Dynamics Dashboard",
            height=1000,
            showlegend=False
        )
        
        return fig

# Initialize FX Engine
fx_engine = QuantumFXEngine()

# Load and analyze data
fx_data = fx_engine.load_fx_data()

print("NERVA Quantum FX Engine initialized successfully")
print("Advanced currency modeling protocols active")

In [ ]:
# Perform advanced volatility clustering analysis
print("Performing volatility clustering analysis...")

# Analyze volatility clustering for major pairs
usd_kes_clustering = fx_engine.analyze_volatility_clustering(fx_data, 'USD_KES')
eur_kes_clustering = fx_engine.analyze_volatility_clustering(fx_data, 'EUR_KES')
gbp_kes_clustering = fx_engine.analyze_volatility_clustering(fx_data, 'GBP_KES')

# Create comprehensive dashboard
print("Generating FX dynamics dashboard...")
fx_dashboard = fx_engine.create_fx_dashboard(fx_data)
fx_dashboard.show()

print("Volatility clustering analysis complete")

# NERVA Quantum FX Dynamics Engine

**Classification:** ALPHA - Authorized Personnel Only  
**System:** National Economic & Risk Visual Analytics  
**Module:** Foreign Exchange Quantum Modeling  
**Clearance Level:** RESTRICTED

---

## Mission Objectives:
- Advanced currency pair modeling with deep learning
- Real-time volatility clustering analysis
- Cross-currency correlation matrices
- Intervention point detection algorithms
- Market microstructure analysis

---